In [ ]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
print("Tensorflow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

In [ ]:
train_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"

train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url), origin=train_dataset_url)

In [ ]:
print("Local copy of dataset file: {}".format(train_dataset_fp))

In [ ]:
!head -n5 {train_dataset_fp}

In [ ]:
column_names= ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

feature_names = column_names[:-1]
label_name = column_names[-1]

print("Features: {}".format(feature_names))
print("Label: {}".format(label_name))

In [ ]:
class_names = ['Iris setosa', 'Iris versicolor', 'Iris virginica']

In [ ]:
batch_size = 32

train_dataset = tf.data.experimental.make_csv_dataset(train_dataset_fp, batch_size, column_names=column_names, label_name=label_name, num_epochs=1)

In [ ]:
train_dataset

In [ ]:
features, labels = next(iter(train_dataset))

In [ ]:
print(features)

In [ ]:
print(labels)

In [ ]:
plt.scatter(features['petal_length'], features['sepal_length'], c = labels, cmap = 'viridis')
plt.xlabel('Petal length')
plt.ylabel('Sepal length')
plt.show()

In [ ]:
plt.scatter(features['sepal_width'], features['petal_width'], c = labels, cmap = 'viridis')
plt.xlabel('sepal_width')
plt.ylabel('petal_width')
plt.show()

In [ ]:
features.values()

In [ ]:
list(features.values())

In [ ]:
tf.stack(list(features.values()))

In [ ]:
tf.stack(list(features.values()), 1)

In [ ]:
def pack_features_vector(features, labels):
    features = tf.stack(list(features.values()), axis=1)
    return features, labels

In [ ]:
train_dataset = train_dataset.map(pack_features_vector)

In [ ]:
features, labels = next(iter(train_dataset))

In [ ]:
print(features)

In [ ]:
print(labels)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, tf.nn.relu, input_shape=(4, )),
    tf.keras.layers.Dense(10, tf.nn.relu),
    tf.keras.layers.Dense(3)
])

In [ ]:
predictions = model(features)

In [ ]:
predictions[:5]

In [ ]:
tf.nn.softmax(predictions[:5])

In [ ]:
print("Prediction: {}".format(tf.argmax(predictions, 1)))
print("Label:      {}".format(labels))

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
def loss(model, x, y, training):
    y_ = model(x, training = training)

    return loss_object(y, y_)

In [ ]:
l = loss(model, features, labels, False)
print(l)

In [ ]:
def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets, True)

    return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [ ]:
optimizer = tf.keras.optimizers.SGD(0.01)

In [ ]:
loss_value, grads = grad(model, features, labels)

In [ ]:
loss_value

In [ ]:
grads

In [ ]:
print("Step: {}, Loss: {}".format(optimizer.iterations.numpy(), loss_value.numpy()))

In [ ]:
optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [ ]:
loss_value, grads = grad(model, features, labels)

In [ ]:
print("Step: {}, Loss: {}".format(optimizer.iterations.numpy(), loss_value.numpy()))

In [ ]:
train_loss_results = []
train_accuracy_results = []

num_epochs = 201

for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

    for x, y in train_dataset:
        loss_value, grads = grad(model, x, y)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        epoch_loss_avg.update_state(loss_value)
        epoch_accuracy.update_state(y, model(x, training = True))
    
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())

    if epoch % 50 == 0:
        print("Epoch: {:03d}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch, epoch_loss_avg.result(), epoch_accuracy.result()))

In [ ]:
fig, axes = plt.subplots(2, 1, True, figsize = (12,8))
fig.suptitle('Training Metrics')

axes[0].set_ylabel('Loss')
axes[0].plot(train_loss_results)

axes[1].set_ylabel('Acc')
axes[1].set_xlabel('Epochs')
axes[1].plot(train_accuracy_results)

plt.show()

In [ ]:
test_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv"

In [ ]:
os.path.basename(test_url)

In [ ]:
test_fp = tf.keras.utils.get_file(fname=os.path.basename(test_url), origin=test_url)

In [ ]:
test_dataset = tf.data.experimental.make_csv_dataset(test_fp, batch_size, column_names=column_names, label_name = 'species', num_epochs=1, shuffle=False)

In [ ]:
test_dataset = test_dataset.map(pack_features_vector)

In [ ]:
test_acc = tf.keras.metrics.Accuracy()

for (x, y) in test_dataset:
    logits = model(x, training=False)
    prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
    test_acc(prediction, y)

print("Test acc: {}".format(test_acc.result()))

In [ ]:
tf.stack([y, prediction], 1)

In [ ]:
predict_dataset = tf.convert_to_tensor([
    [5.1, 3.3, 1.7, 0.5,],
    [5.9, 3.0, 4.2, 1.5,],
    [6.9, 3.1, 5.4, 2.1] 
])

In [ ]:
predictions = model(predict_dataset, training=False)

In [ ]:
predictions

In [ ]:
for i, logits in enumerate(predictions):
    class_idx = tf.argmax(logits).numpy()
    p = tf.nn.softmax(logits)[class_idx]
    name = class_names[class_idx]

    print("Example {}: prediction: {} ({:.4f})".format(i, name, p))